# Random Search

Random Search (RS) is one of the easiest optimization algorithms to understand: pick a random hypothesis, and keep only the best! It isn't technically Bayesian since the next guess has nothing to do with the previous guess, but it's a start. RS is a nice benchmark against which to compare other algorithms, since it's computationally cheap and takes little effort to implement: any algorithm which doesn't beat RS is probably not worth dealing with!

In [1]:
import random as r

## Objective Functions

In this notebook we'll test the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

If you want a different objective function, check out Objectives.ipynb.

In [2]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

## Random Search

In [3]:
class RandomSearch:
    def __init__(
        self,
        ranges
    ):
        self.ranges = ranges
        
    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)
    
    def optimize(self,
                objectiveFunction,
                numIterations = 25000):
        bestX = None
        bestY = -float("inf")
        for i in range(numIterations):
            newX = self.generateRandomHypothesis()
            newY = objectiveFunction(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX)
rs = RandomSearch([[1, 120], [1, 180]])
print(rs.optimize(SEMF))

[28.062189780755695, 34.105709269503244]
